In [ ]:
!pip install langchain openai
!pip install gradio
!pip install google-search-results

In [ ]:
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import ast
import gradio as gr
import pandas as pd
import time
from datetime import datetime
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import os
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [ ]:
response_schema = [
    ResponseSchema(name = "Review Summary",description="A short summary summarising all the important details present in review text and limit it to 3 sentences"),
    ResponseSchema(name = "Sentiment", description="Overall sentiment of review text"),
    ResponseSchema(name = "Pain Points", description = "Issues faced by customers reported in reviews limited to 1 sentence"),
    ResponseSchema(name = "Actions", description = "Corrective suggestions that can be taken to improve customer experience limited to 1 sentence"),
    ResponseSchema(name = "Aspects", description = "List of significant aspects and their sentiment")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()

In [ ]:
prompt = ChatPromptTemplate(
    messages = [
        HumanMessagePromptTemplate.from_template(
            """
            Assume that you are an e-commerce subject matter expert for both English & Japanese marketplaces.
            {persona} reviews will be provided
            % REVIEWS : {review} corresponding to {category}

            Please go through review text and determine the following :

            - Short review summary with no loss of information limited to 3 sentences
            - Determine the overall sentiment
            - List of pain points / issues faced by customer
            - Action items / corrective actions that can be taken to address these issues
            - Important aspects & their sentiment

            Output of the model should follow the structure defined below
            {format_instructions}

            If review text contains irrelevant information or misleading information then please don't consider such sentences.
            Model outputs should be in {language} language only and it must be very polite.
            Please don't try to make up answers. If you don't know the answer to any of these then please output
            "Oops, I'm not sure"
            """
        )
    ],
    input_variables=['review','category','persona','language'],
    partial_variables = {"format_instructions": format_instructions}
)


# Review Analysis thorugh UI

In [ ]:
key = ""
def get_insights(persona,reviews,category,language,model,temperature):
    query = prompt.format_prompt(review=reviews,category=category,persona=persona,language=language)
    # print(query)
    print(persona,category,language,model,temperature)
    query = prompt.format_prompt(review=reviews,persona=persona,language=language,category=category)
    chat_model = ChatOpenAI(temperature = temperature,model_name = model,openai_api_key=key)
    response = chat_model(query.to_messages())
    return response.content.replace("```json","").replace("```","")

In [ ]:
# theme = gr.themes.Base(primary_hue="red",secondary_hue="blue",neutral_hue="green",
#                       text_size="sm",)
theme = gr.themes.Soft()

# Refer : https://www.gradio.app/guides/theming-guide

In [ ]:
demo = gr.Interface(
    fn=get_insights,
    inputs = [
        gr.Dropdown(
            ["Consumer","Seller","E-Commerce Platform","Manufacturer"], label="Persona", info="Select the appropriate persona"),
        gr.Textbox(label="Reviews",lines=3,info="Please enter review text"),
        gr.Dropdown(
            ["Fashion","Shoes","Electronics","Home & Kitchen"], label="Category", info="Select the appropriate category"),
        gr.Textbox(label="Language",info="Model Response Language like English,Japanese,French"),
        gr.Dropdown(
            ["gpt-3.5-turbo","gpt-3.5-turbo-16k","gpt-4","gpt-4-32k"], label="Model", info="Select the model of your choice"),
         gr.Slider(
            label="Temperature",minimum=0.0,maximum=1.0,value=0.3, info="Control the degree of randomness of the model"
        ),
        
    ],
    outputs = [
        gr.Textbox(label="Result",lines=7, info="Model Response")
    ],
    title="Review Insigths Generation",
    description="Review Signals",
    allow_flagging="manual",
    theme = theme,
    flagging_options=["incorrect", "ambiguous"]
)

In [ ]:
demo.queue().launch(share=True)

# File Batch Processing - Reviews

In [ ]:
def get_insights(file_name,column_name,persona,category,language,model,temperature):
    df = pd.read_csv(file_name)
    output_file_name = "processed_"+str(datetime.today().strftime('%Y-%m-%d'))+"_"+file_name
    df['model_output'] = None
    start_time = time.time()
    for i in range(df.shape[0]):
        query = prompt.format_prompt(review=df[column_name].iloc[i],category=category,persona=persona,language=language)
        chat_model = ChatOpenAI(temperature = temperature,model_name = model,openai_api_key=key)
        response = chat_model(query.to_messages())
        response = response.content.replace("```json","").replace("```","")
        df['model_output'].iloc[i] = response
    duration = str(time.time() - start_time)+" S"
    output_file_name = "processed_"+str(datetime.today().strftime('%Y-%m-%d'))+".csv"
    df.to_csv(output_file_name,index=False)
    return {"ouput_file_name":output_file_name,
           "No of processed rows":df.shape[0],
           "No of errored rows":0,
           "Execution Time":duration,
           "Average Time":str((time.time() - start_time)/df.shape[0])+" S" }


In [ ]:
demo = gr.Interface(
    fn=get_insights,
    inputs = [
        gr.Textbox(label="File Name", info= "Enter the file path"),
        gr.Textbox(label="Column Name", info="Specify the coulmn name of the target review field"),
        gr.Dropdown(
            ["Consumer","Seller","E-Commerce Platform","Manufacturer"], label="Persona", info="Select the appropriate persona"),
        gr.Dropdown(
            ["Fashion","Shoes","Electronics","Home & Kitchen"], label="Category", info="Select the appropriate category"),
        gr.Textbox(label="Language", info="Model Response Language like English,Japanese,French"),
        gr.Dropdown(
            ["gpt-3.5-turbo","gpt-3.5-turbo-16k","gpt-4","gpt-4-32k"], label="Model", info="Select the model of your choice"),
         gr.Slider(
            label="Temperature",minimum=0.0,maximum=1.0,value=0.3, info="Control the degree of randomness of the model"
        ),
        
    ],
    outputs = [
        gr.Textbox(label="Result",lines=7, info="Model Response")
    ],
    title="Batch Review Insigths Generation",
    description="Review Signals",
    allow_flagging="manual",
    theme=theme
)

In [ ]:
demo.queue().launch(share=True)

# Gen AI Review Based Product Recommendation

In [ ]:
os.environ['OPENAI_API_KEY']="""
GOOGLE_CSE_ID=""
GOOGLE_API_KEY = ""

In [ ]:
llm = OpenAI(temperature=0, )
search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY,google_cse_id=GOOGLE_CSE_ID)
toolkit = [
    Tool(
        name = "Search",
         func = search.run,
         description="useful for when you need to search google for answers"
    ),]

In [ ]:
agent = initialize_agent(toolkit,llm,agent='zero-shot-react-description', verbose=True)

In [ ]:
def get_web_result(query):
    result = agent.run(query)
    return result
    

In [ ]:
demo = gr.Interface(
    fn=get_web_result,
    inputs = [
        gr.Textbox(label="Query", lines = 7,info="Enter query"),   
    ],
    outputs = [
        gr.Textbox(label="Result",lines=7,info="Model Response")
    ],
    title="Turning Reviews into Recommendations, Powered by GenAI.",
    description="Review Signals",
    allow_flagging="manual",
    flagging_options=["incorrect", "ambiguous"],
    theme = gr.themes.Soft(),
    examples = ["I am interested to buy samsung s23 ultra. Find phones similar to Samsung s23 ultra. Summarise each of the product reviews of similar phones, determine sentiment for each of the summarised review. Recommend the one I should buy with proper justification as to how this product outperforms the competitor phones."]
)

In [ ]:
demo.queue().launch(share=True)

# URL Based Review Insights

In [ ]:
from bs4 import BeautifulSoup
import requests
import gradio as gr

In [ ]:
def get_reviews(url):
    reviews = ''
    try:
        # Send an HTTP GET request and parse the HTML content
        response = requests.get(url)
        response.raise_for_status()  # Check for any HTTP errors

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Use the CSS selector to find elements and extract values
        selected_elements = soup.select(".a-size-base.review-text.review-text-content")

        if selected_elements:
            for element in selected_elements:
                var = element.text.replace("\n","").strip()
                reviews = reviews + var
                print(var)  # Print the text content of the selected elements
        else:
            print(f"No elements found with CSS selector: {css_selector}")
        return var

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
key = ""
def get_insights(persona,page_url,category,language,model,temperature):
    reviews = get_reviews(page_url)
    query = prompt.format_prompt(review=reviews,category=category,persona=persona,language=language)
    print(query)
    print(persona,category,language,model,temperature)
    query = prompt.format_prompt(review=reviews,persona=persona,language=language,category=category)
    chat_model = ChatOpenAI(temperature = temperature,model_name = model,openai_api_key=key)
    response = chat_model(query.to_messages())
    return response.content.replace("```json","").replace("```","")

In [ ]:
theme = gr.themes.Soft()
demo = gr.Interface(
    fn=get_insights,
    inputs = [
        gr.Dropdown(
            ["Consumer","Seller","E-Commerce Platform","Manufacturer"], label="Persona", info="Select the appropriate persona"),
        gr.Textbox(label="Page Url",lines=2,info="Enter Page URL"),
        gr.Dropdown(
            ["Fashion","Shoes","Electronics","Home & Kitchen"], label="Category", info="Select the appropriate category"),
        gr.Textbox(label="Language",info="Model Response Language like English,Japanese,French"),
        gr.Dropdown(
            ["gpt-3.5-turbo","gpt-3.5-turbo-16k","gpt-4","gpt-4-32k"], label="Model", info="Select the model of your choice"),
         gr.Slider(
            label="Temperature",minimum=0.0,maximum=1.0,value=0.3, info="Control the degree of randomness of the model"
        ),
        
    ],
    outputs = [
        gr.Textbox(label="Result",lines=7, info="Model Response")
    ],
    title="Review Insigths Generation",
    description="Review Signals",
    allow_flagging="manual",
    theme = theme,
    flagging_options=["incorrect", "ambiguous"]
)

In [ ]:
demo.queue().launch(share=True)